## команды запуска среды

In [4]:
import sys, os
spark = ## старт вашей спарк сессии на машине

:: loading settings :: file = /data/home/iianishhe1/!WorkSpace/BNKSCOR-11043_check_feat_for_fs/python_3_9/lib/python3.9/site-packages/mtspark/config/ivysettings.xml


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/07/15 09:54:16 WARN HiveConf: HiveConf of name hive.warehouse.data.skiptrash does not exist
25/07/15 09:54:16 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
25/07/15 09:54:16 WARN Client: Exception encountered while connecting to the server 
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyException): Operation category READ is not supported in state standby
	at org.apache.hadoop.security.SaslRpcClient.saslConnect(SaslRpcClient.java:376)
	at org.apache.hadoop.ipc.Client$Connection.setupSaslConnection(Client.java:623)
	at org.apache.hadoop.ipc.Client$Connection.access$2300(Client.java:414)
	at org.apache.hadoop.ipc.Client$Connection$2.run(Client.java:843)
	at org.apache.hadoop.ipc.Client$Connection$2.run(Client.java:839)
	at java.security.AccessController.doPrivileged(Native Method)
	at javax.security.auth.Subject.doAs(Subject.java:422)
	at org.apache.hadoop.security.U

## материал семинара

In [27]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window ## пр-во методов под оконные ф-ии, но мы в семинаре их не рассмортим

In [6]:
## учебный пример фрема данных на семинар

In [7]:
spark ## смотрим на описание запущенной мной спарк сессии

In [8]:
## здесь создаем тестовый пример набора данных, с которым будем работать в рамках семинара

sc = spark.sparkContext

test_data = [
{"name":"Moscow", "country":"Russia", "continent": "Europe", "population": 100_000_000},
{ "name":"Madrid", "country":"Spain" },
{ "name":"Paris", "country":"France", "continent": "Europe", "population" : 205_000_000},
{ "name":"Berlin", "country":"Germany", "continent": "Europe", "population": 140_000_008},
{ "name":"Barselona", "country":"Spain", "continent": "Europe" },
{ "name":"Cairo", "country":"Egypt", "continent": "Africa", "population": 45_000_001 },
{ "name":"Cairo", "country":"Egypt", "continent": "Africa", "population": 45_000_001 },
]

## создаем rdd
rdd = sc.parallelize(test_data)

## создаем DataFrame абстракцию поверх rdd
df = spark.read.json(rdd).localCheckpoint()

df

DataFrame[continent: string, country: string, name: string, population: bigint]

In [72]:
!pwd ## команда для получения пути до вашей home папки и места запуска ноутбука

/data/home/iianishhe1/DE_Sync_2023_2024


In [63]:
!hdfs dfs -ls -h /user/iianishhe1/ ## здесь смотрим на путь до моей юзер папки на HDFS!!!

Found 5 items
drwx------   - iianishhe1 hdfs          0 2025-07-03 03:00 /user/iianishhe1/.Trash
drwxr-xr-x   - iianishhe1 hdfs          0 2025-07-03 16:00 /user/iianishhe1/.sparkStaging
drwx------   - iianishhe1 hdfs          0 2024-07-19 18:25 /user/iianishhe1/.staging
-rw-r--r--   3 iianishhe1 hdfs        232 2025-07-03 17:37 /user/iianishhe1/population.csv
drwxr-xr-x   - iianishhe1 hdfs          0 2025-07-03 17:42 /user/iianishhe1/population_2.csv


In [73]:
# чтение файла по пути папки под юзером на HDFS

read_df = (
    spark
    .read
    .format("csv")
    .options(
        header=True,
        inferSchema=True
    )
    .load("/user/iianishhe1/population.csv")

)

read_df <- # сформированный DataFrame

DataFrame[continent: string, country: string, name: string, population: double]

In [75]:
read_df.show(vertical=True) # вызываем action, смотрим данные в вертикальном разрезе

-RECORD 0------------------
 continent  | Europe       
 country    | Russia       
 name       | Moscow       
 population | 1.0E8        
-RECORD 1------------------
 continent  | null         
 country    | Spain        
 name       | Madrid       
 population | null         
-RECORD 2------------------
 continent  | Europe       
 country    | France       
 name       | Paris        
 population | 2.05E8       
-RECORD 3------------------
 continent  | Europe       
 country    | Germany      
 name       | Berlin       
 population | 1.40000008E8 
-RECORD 4------------------
 continent  | Europe       
 country    | Spain        
 name       | Barselona    
 population | null         
-RECORD 5------------------
 continent  | Africa       
 country    | Egypt        
 name       | Cairo        
 population | 4.5000001E7  
-RECORD 6------------------
 continent  | Africa       
 country    | Egypt        
 name       | Cairo        
 population | 4.5000001E7  



In [86]:
#запись файла по пути до папки юзера на HDFS

(
    read_df
    .write
    .format("parquet")
    .mode("overwrite")
    .save("/user/iianishhe1/population_parquet")
)

In [78]:
!hdfs dfs -ls -h /user/iianishhe1/population_parquet/ # тут наблюдаем за тем как файл успешно записался

Found 2 items
-rw-r--r--   3 iianishhe1 hdfs          0 2025-07-03 19:48 /user/iianishhe1/population_parquet/_SUCCESS
-rw-r--r--   3 iianishhe1 hdfs      1.5 K 2025-07-03 19:48 /user/iianishhe1/population_parquet/part-00000-20b11fbc-7f3c-44a3-b6fd-c48abc1f6d6e-c000.snappy.parquet


In [79]:
!hdfs dfs -cat /user/iianishhe1/population_parquet/* # тут попытались вывести файл на чтение в консоль
# но формат паркет хранит данные бинарно, так что просто так их не прочитать

PAR1(,���<  L   Europe   Africa ����  	    }0bd�Ŭ<
  1H   Russia   SpaindFrance   Germany   Egypt �ϐ�  (   �zx����<  =$   Moscow
4adrid   ParislBerlin	   Barselona   Cairo ��ǜ	  (   ��@B����<   8    �חA   �p�0v��A   *u�A �ʆ�	  
$   m�AfricaEurope EgyptSpain  	BarselonaParis     *u�A   �p�A ZD   �H   �H   �F   \Hspark_schema %	continent% L   %country% L   %name% L   
%
population L&Z5	continent��&Z&6(EuropeAfrica ,    ��6 &�5country��&�&�6 (SpainEgypt ,    ��2 &�5name��&�&�6 (Paris	Barselona ,    ��: &�
5
population��&�&�   �p�A   *u�A(   �p�A   *u�A ,    ��> 

In [82]:
## Создание DataFrame от локального файла, он лежит рядом с ноутбуком, не на HDFS!!!
pdf = pd.read_csv("population.csv")
pdf

,continent,country,name,population
0,Europe,Russia,Moscow,100000000.000
1,NaN,Spain,Madrid,NaN
2,Europe,France,Paris,205000000.000
3,Europe,Germany,Berlin,140000008.000
4,Europe,Spain,Barselona,NaN
5,Africa,Egypt,Cairo,45000001.000
6,Africa,Egypt,Cairo,45000001.000


In [83]:
test_pdf = (
    spark.createDataFrame(pdf) ## создание DataFrame от локально pandas объекта
)

test_pdf

DataFrame[continent: string, country: string, name: string, population: double]

In [85]:
test_pdf.toPandas()

,continent,country,name,population
0,Europe,Russia,Moscow,100000000.000
1,None,Spain,Madrid,NaN
2,Europe,France,Paris,205000000.000
3,Europe,Germany,Berlin,140000008.000
4,Europe,Spain,Barselona,NaN
5,Africa,Egypt,Cairo,45000001.000
6,Africa,Egypt,Cairo,45000001.000


Смотрим на методы взаимодействия с DataFrame:
- select
- filter
- dropDuplicates
- na.fill
- replace
- orderBy

Триггерные действия:
- show
- save
- saveAsTable - просто рассказать

документация со всем, что может пригодиться:
- методы к DataFrame - https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html
- методы из sql.functions - https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

- Посмотрим .explain() и на сами планы
- alias и когда он нужен
- join и как выглядят их планы вычислений

Будем переодически смотреть на UI и как там все это выглядит

Полезная ссылка - https://habr.com/ru/articles/901078/ - тут написано про Catalyst и как он оптимизирует ваши запросы к данным

In [ ]:
## Важно добавить, в sql.functions методах для указания колонок, над которыми данные хотите применить методы
## указывать колонки через перечисление string переменных, либо через метод F.col() - к которому также можно применять свои методы
## Второй подход более предпочтительный, так как к объектам колонок F.col() можно применить дополнительные методы

In [213]:
## Посмотреть как выглядит реализация какой-то задачи на SQL

df.createOrReplaceTempView("test_test") ## создание метки view для обращения к ней через "FROM test_test" 


## Далее всем знакомый вам SQL код
spark.sql("""
    select *
    
    from test_test as a join test_test as b on a.name = b.name
    
    where a.name = 'Moscow'

""").explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('a.country = Russia)
   +- 'Join Inner, ('a.name = 'b.name)
      :- 'SubqueryAlias a
      :  +- 'UnresolvedRelation [test_test], [], false
      +- 'SubqueryAlias b
         +- 'UnresolvedRelation [test_test], [], false

== Analyzed Logical Plan ==
continent: string, country: string, name: string, population: bigint, continent: string, country: string, name: string, population: bigint
Project [continent#189, country#190, name#191, population#192L, continent#2473, country#2474, name#2475, population#2476L]
+- Filter (country#190 = Russia)
   +- Join Inner, (name#191 = name#2475)
      :- SubqueryAlias a
      :  +- SubqueryAlias test_test
      :     +- View (`test_test`, [continent#189,country#190,name#191,population#192L])
      :        +- LogicalRDD [continent#189, country#190, name#191, population#192L], false
      +- SubqueryAlias b
         +- SubqueryAlias test_test
            +- View (`test_test`, [continent#2473,country#24

In [11]:
## Реализация запроса выше через DataFrame методы

(
    df
    .select("*")
    .join(
        df,
        on=["name"],
        how="inner"
    )
    .filter(F.col("name") == "Moscow")
    .show()
)

## реализовали .select("*") - всех колонок, но так делать не стоит!!! явно указывайте список нужных колонок
## .join() - джоин первого датафрейма ко второму по нужной колонке и с условием inner
## добавление фильтра по колонке name с опр значениями в ней

+------+---------+-------+----------+---------+-------+----------+
|  name|continent|country|population|continent|country|population|
+------+---------+-------+----------+---------+-------+----------+
|Moscow|   Europe| Russia| 100000000|   Europe| Russia| 100000000|
+------+---------+-------+----------+---------+-------+----------+



In [13]:
## разбираем отдельно методы и их планы запросов:
    
(
    df
    .select("name", "continent", "country") ## метод выбора нужных колонок из дата фрейма
    .show()
)

(
    df
    .select("name", "continent", "country")
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## сначала делает скан  коллекции а после вызывает метод Project для выделения нужных колонок из данных
## ПЛАНЫ ЗАПРОСОВ ЧИТАЮТСЯ СНИЗУ ВВЕРХ

## отличия каждого из планов более подробно разобраны на лекции, в блоке про их чтение и определение

+---------+---------+-------+
|     name|continent|country|
+---------+---------+-------+
|   Moscow|   Europe| Russia|
|   Madrid|     null|  Spain|
|    Paris|   Europe| France|
|   Berlin|   Europe|Germany|
|Barselona|   Europe|  Spain|
|    Cairo|   Africa|  Egypt|
|    Cairo|   Africa|  Egypt|
+---------+---------+-------+

== Parsed Logical Plan ==
'Project ['name, 'continent, 'country]
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Analyzed Logical Plan ==
name: string, continent: string, country: string
Project [name#8, continent#6, country#7]
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Optimized Logical Plan ==
Project [name#8, continent#6, country#7]
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Physical Plan ==
*(1) Project [name#8, continent#6, country#7]
+- *(1) Scan ExistingRDD[continent#6,country#7,name#8,population#9L]



In [14]:
(
    df
    .filter(F.col("name") == "Moscow") ## условие фильтрации данных по значению из колонки, аналог WHERE в SQL
    .show()
)

(
    df
    .filter(F.col("name") == "Moscow")
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## сначала делает скан  коллекции а после применяется метод Filter для проверки того условия,
## что мы указали в методах к DataFrame

+---------+-------+------+----------+
|continent|country|  name|population|
+---------+-------+------+----------+
|   Europe| Russia|Moscow| 100000000|
+---------+-------+------+----------+

== Parsed Logical Plan ==
'Filter ('name = Moscow)
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Analyzed Logical Plan ==
continent: string, country: string, name: string, population: bigint
Filter (name#8 = Moscow)
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Optimized Logical Plan ==
Filter (isnotnull(name#8) AND (name#8 = Moscow))
+- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Physical Plan ==
*(1) Filter (isnotnull(name#8) AND (name#8 = Moscow))
+- *(1) Scan ExistingRDD[continent#6,country#7,name#8,population#9L]



In [16]:
(
    df
    .show() ## в таком вызове в данных видим полные дубли по строкам, конкретно в данных по Африке
)

(
    df
    .dropDuplicates() ## метод выявления дублей и их удалние, если аргументов нет - дубли ищутся по всем строкам
    .show()
)

(
    df
    .dropDuplicates() 
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## сначала скан коллекции а после связка трех методов - HashAggregate, Exchange hashpartitioning и снова HashAggregate
## посянение почему так метод реализуется на уровне движка лучше в записи семинара
## я там явно проговариваю логику работы оптимизатора

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|   Europe| Russia|   Moscow| 100000000|
|     null|  Spain|   Madrid|      null|
|   Europe| France|    Paris| 205000000|
|   Europe|Germany|   Berlin| 140000008|
|   Europe|  Spain|Barselona|      null|
|   Africa|  Egypt|    Cairo|  45000001|
|   Africa|  Egypt|    Cairo|  45000001|
+---------+-------+---------+----------+

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|   Europe| Russia|   Moscow| 100000000|
|   Europe| France|    Paris| 205000000|
|   Europe|Germany|   Berlin| 140000008|
|   Europe|  Spain|Barselona|      null|
|   Africa|  Egypt|    Cairo|  45000001|
|     null|  Spain|   Madrid|      null|
+---------+-------+---------+----------+

== Parsed Logical Plan ==
Deduplicate [continent#6, country#7, name#8, population#9L]
+- LogicalRDD [continent#6, country#7, name#8, popu

In [18]:
(
    df
    .show() 
)

(
    df
    .orderBy(F.asc("continent")) ## метод сортировки данных, обычно полезен для визуализации какой-то части данных в вашем выводе
    .show()
)

(
    df
    .orderBy(F.asc("continent"))
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## Важное замечание - тут вызывается Exchange rangepartitioning, который сильно хуже оптимизирован, чем Exchange hashpartitioning
## Метод сортировки данных стоит использовать только для промежуточных вычислений, если в задаче обработки данных у вас
## явно ничего нет про сортировку фрейма данных
подробнее про виды partitioning стоит прочитать в доке по ссылкам выше

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|   Europe| Russia|   Moscow| 100000000|
|     null|  Spain|   Madrid|      null|
|   Europe| France|    Paris| 205000000|
|   Europe|Germany|   Berlin| 140000008|
|   Europe|  Spain|Barselona|      null|
|   Africa|  Egypt|    Cairo|  45000001|
|   Africa|  Egypt|    Cairo|  45000001|
+---------+-------+---------+----------+

+---------+-------+---------+----------+
|continent|country|     name|population|
+---------+-------+---------+----------+
|     null|  Spain|   Madrid|      null|
|   Africa|  Egypt|    Cairo|  45000001|
|   Africa|  Egypt|    Cairo|  45000001|
|   Europe| Russia|   Moscow| 100000000|
|   Europe|Germany|   Berlin| 140000008|
|   Europe| France|    Paris| 205000000|
|   Europe|  Spain|Barselona|      null|
+---------+-------+---------+----------+

== Parsed Logical Plan ==
'Sort ['continent ASC NULLS FIRST], true
+- LogicalRDD [continent#6, c

In [19]:
## джойны и их планы запросов
(
    df
    .join(
        df,
        on=["name"],
        how="inner",
    )
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## Видим выбор SortMergeJoin подхода в движке
## и предварительные сортировки ключей "Sort [name#8 ASC NULLS FIRST], false, 0" чтобы раскидать их по нодам

== Parsed Logical Plan ==
'Join UsingJoin(Inner,Buffer(name))
:- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- LogicalRDD [continent#259, country#260, name#261, population#262L], false

== Analyzed Logical Plan ==
name: string, continent: string, country: string, population: bigint, continent: string, country: string, population: bigint
Project [name#8, continent#6, country#7, population#9L, continent#259, country#260, population#262L]
+- Join Inner, (name#8 = name#261)
   :- LogicalRDD [continent#6, country#7, name#8, population#9L], false
   +- LogicalRDD [continent#259, country#260, name#261, population#262L], false

== Optimized Logical Plan ==
Project [name#8, continent#6, country#7, population#9L, continent#259, country#260, population#262L]
+- Join Inner, (name#8 = name#261)
   :- Filter isnotnull(name#8)
   :  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
   +- Filter isnotnull(name#261)
      +- LogicalRDD [continent#259, country#2

In [20]:
## джойны и их планы запросов
(
    df
    .join(
        F.broadcast(df),
        on=["name"],
        how="inner",
    )
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## Видим выбор BroadcastHashJoin, потому что к одному из фреймов явно применили Broadcast метод
## про этот метод есть пояснение в лекции
## также важно: видно что здесь нет этапа сортировки ключей, даже на втором фрейме данных
## отсуствие этого этапа рассчета для джойна и дает нам оптимизацию по времени его работы

== Parsed Logical Plan ==
'Join UsingJoin(Inner,Buffer(name))
:- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- ResolvedHint (strategy=broadcast)
   +- LogicalRDD [continent#270, country#271, name#272, population#273L], false

== Analyzed Logical Plan ==
name: string, continent: string, country: string, population: bigint, continent: string, country: string, population: bigint
Project [name#8, continent#6, country#7, population#9L, continent#270, country#271, population#273L]
+- Join Inner, (name#8 = name#272)
   :- LogicalRDD [continent#6, country#7, name#8, population#9L], false
   +- ResolvedHint (strategy=broadcast)
      +- LogicalRDD [continent#270, country#271, name#272, population#273L], false

== Optimized Logical Plan ==
Project [name#8, continent#6, country#7, population#9L, continent#270, country#271, population#273L]
+- Join Inner, (name#8 = name#272), rightHint=(strategy=broadcast)
   :- Filter isnotnull(name#8)
   :  +- LogicalRDD [continent#6, coun

In [21]:
## джойны и их планы запросов
(
    df.alias("a")
    .join(
        F.broadcast(df.alias("b")),
        on=F.col("a.name") != F.col("b.name"),
        how="inner",
    )
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## Видим выбор BroadcastNestedLoopJoin по причине того,
## что условие джйона не вписывается в проверку ключей на равенство + применем Broadcast к одному из фреймов данных
## Сортировки ключей тут также нет
## Но join данных по частям на каждом воркере будет выполняться через вложенный цикл, из-за сложности условия

== Parsed Logical Plan ==
Join Inner, NOT (name#8 = name#283)
:- SubqueryAlias a
:  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- ResolvedHint (strategy=broadcast)
   +- SubqueryAlias b
      +- LogicalRDD [continent#281, country#282, name#283, population#284L], false

== Analyzed Logical Plan ==
continent: string, country: string, name: string, population: bigint, continent: string, country: string, name: string, population: bigint
Join Inner, NOT (name#8 = name#283)
:- SubqueryAlias a
:  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- ResolvedHint (strategy=broadcast)
   +- SubqueryAlias b
      +- LogicalRDD [continent#281, country#282, name#283, population#284L], false

== Optimized Logical Plan ==
Join Inner, NOT (name#8 = name#283), rightHint=(strategy=broadcast)
:- Filter isnotnull(name#8)
:  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- Filter isnotnull(name#283)
   +- LogicalRDD [continent#281, country

In [24]:
## джойны и их планы запросов

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1") ## для след примера выключим автоматический Broadcast
## небольших фреймов данных

(
    df.alias("a")
    .join(
        df.alias("b"),
        on=F.col("a.name") != F.col("b.name"),
        how="inner",
    )
    .explain(True)
)

## по плану запроса видно, что Physical Plan (физический движок с вызовом конкретных методов)
## Видим выбор CartesianProduct, самый "долгий" из физических подходов по вычислению работы джойна
## В силу сложности условия джойна + невозможности применить Broadcast (мы его буквально отключили)
## был выбран CartesianProduct метод
## оптимизаций тут нет, данные распределяются на все возможные паросочетания ключей
## раскидываются на ноды по хэшу и после проверяются по условию на правильно объединения данных

== Parsed Logical Plan ==
Join Inner, NOT (name#8 = name#343)
:- SubqueryAlias a
:  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- SubqueryAlias b
   +- LogicalRDD [continent#341, country#342, name#343, population#344L], false

== Analyzed Logical Plan ==
continent: string, country: string, name: string, population: bigint, continent: string, country: string, name: string, population: bigint
Join Inner, NOT (name#8 = name#343)
:- SubqueryAlias a
:  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- SubqueryAlias b
   +- LogicalRDD [continent#341, country#342, name#343, population#344L], false

== Optimized Logical Plan ==
Join Inner, NOT (name#8 = name#343)
:- Filter isnotnull(name#8)
:  +- LogicalRDD [continent#6, country#7, name#8, population#9L], false
+- Filter isnotnull(name#343)
   +- LogicalRDD [continent#341, country#342, name#343, population#344L], false

== Physical Plan ==
CartesianProduct NOT (name#8 = name#343)
:- *(1) Filter i

In [26]:
## ниже план запроса методов split и explode
## которые превращают строковую колонку в СПИСОК символов
## а после список символов мы раскидываем на уникальные значения для каждой строки, тем самым
## на каждый эл-т из списка создаем новую строку

(
    df
    .withColumn("name_sep", F.split("name", ""))
    .withColumn("name_exp", F.explode("name_sep"))
    .filter(F.col("name_exp") == "s")
    .explain(True)
)

## Логику отработки этого метода можно увидеть на, Physical Plan

== Parsed Logical Plan ==
'Filter ('name_exp = s)
+- Project [continent#6, country#7, name#8, population#9L, name_sep#375, name_exp#382]
   +- Generate explode(name_sep#375), false, [name_exp#382]
      +- Project [continent#6, country#7, name#8, population#9L, split(name#8, , -1) AS name_sep#375]
         +- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Analyzed Logical Plan ==
continent: string, country: string, name: string, population: bigint, name_sep: array<string>, name_exp: string
Filter (name_exp#382 = s)
+- Project [continent#6, country#7, name#8, population#9L, name_sep#375, name_exp#382]
   +- Generate explode(name_sep#375), false, [name_exp#382]
      +- Project [continent#6, country#7, name#8, population#9L, split(name#8, , -1) AS name_sep#375]
         +- LogicalRDD [continent#6, country#7, name#8, population#9L], false

== Optimized Logical Plan ==
Filter (name_exp#382 = s)
+- Generate explode(name_sep#375), false, [name_exp#382]
   +- Project [c

## Сравнение реализаций задачи над RDD и DataFrame

In [88]:
## пример задачи через RDD на подсчет числа упоминаний стран в данных


    rdd \
    .map(lambda x: x['country'].lower()) \
    .map( lambda x: (x, 1)) \
    .reduceByKey( lambda x,y: x + y) \
    .collect()


[('russia', 1), ('germany', 1), ('spain', 2), ('egypt', 2), ('france', 1)]

In [98]:
## она же через DataFrame API
(
    df
    .groupBy("country")
    .agg(F.count("*"))
    .explain(True)
)

== Parsed Logical Plan ==
'Aggregate ['country], ['country, count(1) AS count(1)#466L]
+- LogicalRDD [continent#189, country#190, name#191, population#192L], false

== Analyzed Logical Plan ==
country: string, count(1): bigint
Aggregate [country#190], [country#190, count(1) AS count(1)#466L]
+- LogicalRDD [continent#189, country#190, name#191, population#192L], false

== Optimized Logical Plan ==
Aggregate [country#190], [country#190, count(1) AS count(1)#466L]
+- Project [country#190]
   +- LogicalRDD [continent#189, country#190, name#191, population#192L], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[country#190], functions=[count(1)], output=[country#190, count(1)#466L])
   +- Exchange hashpartitioning(country#190, 400), ENSURE_REQUIREMENTS, [plan_id=547]
      +- HashAggregate(keys=[country#190], functions=[partial_count(1)], output=[country#190, count#470L])
         +- Project [country#190]
            +- Scan ExistingRDD[continent#189,cou